In [12]:
using Distributions, FileIO, CSV, DataFrames, Revise, Plots, Query, StatsPlots

In [13]:


DATA = CSV.read("/home/khaled/Documents/CSV/MixedData/DataCleanF.csv");

# داده های جدیتر 
##  اگر از این داده استفاده می‌کنید سلول زیر را فعال و سلول بعدی را اجرا کنید در غیر اینصورت از این دو سلول عبور کنید 

In [14]:
#DATA=CSV.read("/home/khaled/Documents/CSV/Dr.Goudarzi/DATAMixedOldDataAndNewData.csv");

In [15]:
function TypeToNumber(v)
    uv = unique(v)
    n = length(v)
    res = zeros(Int, n)
    for i in 1:n
        for j in 1:length(uv)
            if v[i] == uv[j]
                res[i] = j
                break
            end
        end
    end
    return res
end;
DATA.CovNumber = TypeToNumber(DATA.CoverageType);

In [16]:
unique(DATA.Person);

In [6]:
        Person = convert(Array, DATA.Person)
        Times = convert(Array, DATA.Length)
        Cov = convert(Array, DATA.CovNumber)
        Paid = convert(Array, DATA.LossPaid)
        Unique_Coverage = unique(DATA.CovNumber)
        typeof(Unique_Coverage)
        function ClaimTimes(Person, Times, Cov, CovNumber)
            CIndex = (Cov .== CovNumber)
            T1 = Times[CIndex]
            P1 = Person[CIndex]
            L = []
            for i in unique(P1)
                #AA = D1[D1.Person .== i, 2]
                AA = T1[P1.==i]
                if length(AA) > 0
                    append!(L, unique(AA))
                end
            end
            return L
        end


ClaimTimes (generic function with 1 method)

In [17]:

        function ClaimSizes(Person, Times, Paied, Cov, CovNumber)
            CSIndex = (Cov .== CovNumber)
            T1 = Times[CSIndex]
            P1 = Person[CSIndex]
            Paid1 = Paid[CSIndex]
            Cost1 = [] #first claim sevirty#ok
            Cost2 = [] #second claim sevirty#ok
            Cost3 = [] #three or more claim severity
            for i in unique(P1)
                AA = T1[P1.==i]
                uAA = unique(AA)
                if length(uAA) == 1
                    minAA = uAA
                    push!(Cost1, Paid1[(P1.==i).&(T1.==minAA[1])])
                end
                if length(uAA) == 2
                    sAA = sort(uAA)
                    push!(Cost1, Paid1[(P1.==i).&(T1.==sAA[1])])
                    push!(Cost2, Paid1[(P1.==i).&(T1.==sAA[2])])
                end
                if length(uAA) > 2
                    sAA = sort(uAA)
                    push!(Cost1, Paid1[(P1.==i).&(T1.==sAA[1])])
                    push!(Cost2, Paid1[(P1.==i).&(T1.==sAA[2])])
                    for l in sAA[3:end]
                        push!(Cost3, Paid1[(P1.==i).&(T1.==l)])
                    end
                end
            end
            return Cost1, Cost2, Cost3
        end

ClaimSizes (generic function with 1 method)

In [18]:
function Edit(A, B)
    if length(A) >= 2
        D = [sum(A[i]) for i in 1:length(A)]
    else
        D = [sum(B[i]) for i in 1:length(B)]
    end
    return D
end

Edit (generic function with 1 method)

In [19]:
function Edit1(A, B, C, F)
    if length(A) >= 2
        D = C
    else
        D = F
    end
    return D
end

Edit1 (generic function with 1 method)

In [20]:
AgeBound = [[0, 45], [40, 60], [60, 90]]
AgeBoundName = [" رده اول", "  رده دوم", "  رده سوم "]

3-element Array{String,1}:
 " رده اول"
 "  رده دوم"
 "  رده سوم "

In [24]:
for j in 1:3
    DATA2 = DATA[AgeBound[j][1].<DATA.Age.<AgeBound[j][2], :]
    Person = convert(Array, DATA2.Person)
    Times = convert(Array, DATA2.Length)
    Cov = convert(Array, DATA2.CovNumber)
    Paid = convert(Array, DATA2.LossPaid)
    Unique_Coverage = unique(DATA.CovNumber)
    NAME1 = AgeBoundName[j]
    for i in 4:17
        A = ClaimSizes(Person, Times, Paid, Cov, unique(DATA2.CovNumber)[i])
        Plot = density(log10.(abs.([sum(A[1][i]) for i in 1:length(A[1])])),labels = "First arrival", legend = :topright)

        density!(log10.(abs.(Edit(A[2], A[1]))),labels = "Second arrival", legend = :topright)

        density!(log10.(abs.(Edit(A[3], A[1]))),labels = "Third arrival", legend = :topright)
        CovE=unique(DATA2.CoverageType)
        Path = "/home/khaled/Documents/CSV/Dr.Goudarzi/"
        NAME = CovE[i]
        
        savefig(Plot, Path * "Plots/" * NAME * "-" * NAME1 * ".pdf")
    end
end
     
